# Chapter 19: Derivative Valuation Models


In this chapter, we start with option pricing using binomial trees, then move on to the classical Black-Scholes-Merton (BSM) option pricing model. We also cover the basic valuation model for forwards and futures, and finally, we discuss the pricing of credit default swaps (CDSs).

Chapter Objectives

1. Understand binomial option pricing
2. Explore the Black-Scholes-Merton option pricing model
3. Learn how to price forwards and futures
4. Discuss the pricing of credit default swaps

In [1]:
# Stock payoffs in different states
stock_payoffs = {"upstate":120, "downstate":80}

# Strike price is $100
X = 100
# Calculate the option payoffs in the future
option_payoffs = {}
for key, value in stock_payoffs.items():
    option_payoffs[key] = max(value-X,0)
print("the option payoffs are:")   
print(option_payoffs)



the option payoffs are:
{'upstate': 20, 'downstate': 0}


In [2]:
# Create a replicating portfolio
num_stock = 0.5
debt_fv = 40

# Calculate the option payoffs in the future
repli_payoffs = {}
for key, value in stock_payoffs.items():
    repli_payoffs[key] = value*num_stock - debt_fv
print("the replicating portfolio payoffs are:")   
print(repli_payoffs)

# Calculate the cost of the replicating portfolio
stock_price = 100
r = 0.1
repli_price = num_stock*stock_price - debt_fv/(1+r)

# Option price should be the same
option_price = repli_price
print('the cost of the replicating portfolio is', repli_price)
print('the price of the option is', option_price)



the replicating portfolio payoffs are:
{'upstate': 20.0, 'downstate': 0.0}
the cost of the replicating portfolio is 13.63636363636364
the price of the option is 13.63636363636364


In [3]:
# Create a hedging portfolio
num_stock = 0.5
num_option = -1

# Calculate the hedging portfolio payoffs 
hedge_payoffs = {}
for key, value in stock_payoffs.items():
    hedge_payoffs[key] = value*num_stock + \
    option_payoffs[key]*num_option
print("the hedging portfolio payoffs are:")   
print(hedge_payoffs)



the hedging portfolio payoffs are:
{'upstate': 40.0, 'downstate': 40.0}


In [4]:
# One period binomial option pricing
import math

# Parameter values
S0 = 100
u = 1.2
d = 0.8
r = 0.1
T = 1
c_u = 20
c_d = 0

# Calcualte option price
rho = (math.exp(r*T)-d)/(u-d)
c = math.exp(-r*T)*(rho*c_u+(1-rho)*c_d)
print('the option price is', c)



the option price is 13.806503278561621


In [5]:
# Two period binomial option pricing
import math

# Parameter values
S0 = 100
u = 1.2
d = 0.8
r = 0.1
T = 1
c_uu = 44
c_ud = 0
c_du = 0
c_dd = 0

# Calcualte option price at time 1
rho = (math.exp(r*T)-d)/(u-d)
c_u = math.exp(-r*T)*(rho*c_uu+(1-rho)*c_ud)
c_d = math.exp(-r*T)*(rho*c_du+(1-rho)*c_dd)
print('the option price at time 1 on the up node is', round(c_u,2))
print('the option price at time 1 on the down node is', round(c_d,2))  

# Calcualte option price at time 0
c = math.exp(-r*T)*(rho*c_u+(1-rho)*c_d)
print('the option price at time 0 is', round(c,2))


the option price at time 1 on the up node is 30.37
the option price at time 1 on the down node is 0.0
the option price at time 0 is 20.97


In [6]:
from scipy.stats import norm
from numpy import exp, log, sqrt
import numpy as np
from scipy.optimize import minimize

S0 = 45
std_s = 0.2
r = 0.1
T = 0.5
K = 40

def d1(s,r,t,x,std):
    return (log(s/x)+(r+0.5*std**2)*t)/(std*sqrt(t))

def d2(s,r,t,x,std):
    return (log(s/x)+(r-0.5*std**2)*t)/(std*sqrt(t))
    
d_1 = d1(S0, r, T, K, std_s)
d_2 = d2(S0, r, T, K, std_s)

# Print out results
print("the value of d1 is", d_1)
print("the value of d2 is", d_2)  

# Define the call and put option price functions
def c_bsm(s,r,t,x,std):
    d_1 = d1(s,r,t,x,std)
    d_2 = d2(s,r,t,x,std)
    c = S0*norm.cdf(d_1)-K*exp(-r*T)*norm.cdf(d_2)
    return c

def p_bsm(s,r,t,x,std):
    d_1 = d1(s,r,t,x,std)
    d_2 = d2(s,r,t,x,std)
    p = K*exp(-r*T)*norm.cdf(-d_2)-S0*norm.cdf(-d_1)
    return p

# Print out results
c = c_bsm(S0, r, T, K, std_s)
p = p_bsm(S0, r, T, K, std_s)
print("the value of call option is", round(c,2))
print("the value of put option is", round(p,2))


the value of d1 is 1.2571159009255848
the value of d2 is 1.1156945446882756
the value of call option is 7.29
the value of put option is 0.34


In [7]:
# Parameter values
S0 = 600
r = 0.1
T = 1

# Calcualte forward price
forward_price = S0*math.exp(r*T)
print("the forward price is", round(forward_price,2))



the forward price is 663.1


In [8]:
# Forward price when there is dividend payment
import math

S0 = 600
r = 0.1
T = 1
# Dividend payment date
tao = 0.5 
# Dividend amount
D = 30 

# Calcualte forward price
forward_price_d = (S0 - D*math.exp(-r*tao))*math.exp(r*T)
print("the forward price with dividend is", round(forward_price_d,2))



the forward price with dividend is 631.56


In [9]:
# Forward price with storage cost
import math

S0 = 600
r = 0.1
T = 1
# Storage cost
C = S0 * 0.1 

# Calcualte forward price
forward_price_s = (S0 + C)*math.exp(r*T)
print("the forward price with storage cost is", round(forward_price_s,2))



the forward price with storage cost is 729.41


In [10]:
import numpy as np
import pandas as pd
                
dfs= pd.DataFrame(index=[1,2,3,4,5],
                  columns=['yr'])

dfs['yr']=dfs.index

dfs['p_default']=0.03
dfs['p_survive']=0.97


for i in (2,3,4,5):
    dfs.at[i,'p_default']=dfs.at[i-1,'p_survive']*0.03
    dfs.at[i,'p_survive']=dfs.at[i-1,'p_survive']-dfs.at[i,'p_default']  

for i in (1,2,3,4,5):
    dfs.at[i,'e_pv']=dfs.at[i,'p_survive']/1.05**i

dfs['recovery']=0.2
for i in (1,2,3,4,5):
    dfs.at[i,'payoff']=dfs.at[i,'p_default']*(1-dfs.at[i,'recovery'])

for i in (1,2,3,4,5):
    dfs.at[i,'pv_payoff']=dfs.at[i,'payoff']/1.05**i

print(dfs)

e_pv_sum = dfs['e_pv'].sum()
pv_payoff_sum = dfs['pv_payoff'].sum()

CDS_spread = pv_payoff_sum/e_pv_sum

print("the CDS spread is", round(CDS_spread,5))


   yr  p_default  p_survive      e_pv  recovery    payoff  pv_payoff
1   1   0.030000   0.970000  0.923810       0.2  0.024000   0.022857
2   2   0.029100   0.940900  0.853424       0.2  0.023280   0.021116
3   3   0.028227   0.912673  0.788401       0.2  0.022582   0.019507
4   4   0.027380   0.885293  0.728333       0.2  0.021904   0.018021
5   5   0.026559   0.858734  0.672841       0.2  0.021247   0.016648
the CDS spread is 0.02474
